In [3]:
import numpy as np

import pytransform3d.rotations as pr
from pytransform3d.plot_utils import plot_vector

In [6]:
origin = np.array([0, 0, 0])

# basis vectors
x = np.array([1, 0, 0])
y = np.array([0, 1, 0])
z = np.array([0, 0, 1])

# basis vectors as homogeneous coordinates
xh = np.array([1, 0, 0, 1])
yh = np.array([0, 1, 0, 1])
zh = np.array([0, 0, 1, 1])

In [7]:
# Get transformation matrix
def get_rot_x(angle):
    '''
    transformation matrix that rotates a point about the standard X axis
    '''
    Rx = np.zeros(shape=(3, 3))
    Rx[0, 0] = 1
    Rx[1, 1] = np.cos(angle)
    Rx[1, 2] = -np.sin(angle)
    Rx[2, 1] = np.sin(angle)
    Rx[2, 2] = np.cos(angle)
    
    return Rx

def get_rot_y(angle):
    '''
    transformation matrix that rotates a point about the standard Y axis
    '''
    Ry = np.zeros(shape=(3, 3))
    Ry[0, 0] = np.cos(angle)
    Ry[0, 2] = -np.sin(angle)
    Ry[2, 0] = np.sin(angle)
    Ry[2, 2] = np.cos(angle)
    Ry[1, 1] = 1
    
    return Ry

def get_rot_z(angle):
    '''
    transformation matrix that rotates a point about the standard Z axis
    '''
    Rz = np.zeros(shape=(3, 3))
    Rz[0, 0] = np.cos(angle)
    Rz[0, 1] = -np.sin(angle)
    Rz[1, 0] = np.sin(angle)
    Rz[1, 1] = np.cos(angle)
    Rz[2, 2] = 1
    
    return Rz

In [8]:
def create_rotation_transformation_matrix(angles, order):
    '''
    Create a matrix that rotates a vector through the given angles in the given order
    wrt the standard global axes (extrinsic rotation)
    Note: The rotation is carried out anti-clockwise in a left handed axial system
    
    Parameters
    -----------
    angles - list
        list of angles in radians
    order - string
        the order in which to rotate
        
    Returns
    --------
    net - np.ndarray, shape - (3, 3)
        The transformation matrix which carries out the given rotations
    '''
    fn_mapping = {'x': get_rot_x, 'y': get_rot_y, 'z': get_rot_z}
    net = np.identity(3)
    for angle, axis in list(zip(angles, order))[::-1]:
        if fn_mapping.get(axis) is None:
            raise ValueError("Invalid axis")
        R = fn_mapping.get(axis)
        net = np.matmul(net, R(angle))
        
    return net

def create_translation_matrix(offset):
    '''
    Create a transformation matrix that translates a vetor by the given offset
    
    Parameters
    -----------
    offset - np.ndarray, shape - (3,)
        The translation offset
    
    Returns
    ----------
    T - np.ndarray, shape - (4, 4)
        The translation matrix
    '''
    T = np.identity(4)
    T[:3, 3] = offset
    return T

In [9]:
R = create_rotation_transformation_matrix([np.pi/12], 'y')
R_ = np.identity(4)
R_[:3, :3] = R

# create translation transformation matrix
T_ = create_translation_matrix(np.array([0, 28, 0]))

In [10]:
R_, T_

(array([[ 0.96592583,  0.        , -0.25881905,  0.        ],
        [ 0.        ,  1.        ,  0.        ,  0.        ],
        [ 0.25881905,  0.        ,  0.96592583,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0., 28.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]))

In [11]:
def compute_intrinsic_parameter_matrix(f, s, a, cx, cy):
    K = np.identity(3)
    K[0, 0] = f
    K[0, 1] = s
    K[0, 2] = cx
    K[1, 1] = a * f
    K[1, 2] = cy
    
    return K

In [12]:
intrinsic_matrix = compute_intrinsic_parameter_matrix(3.37,0,1,0,28)
intrinsic_matrix

array([[ 3.37,  0.  ,  0.  ],
       [ 0.  ,  3.37, 28.  ],
       [ 0.  ,  0.  ,  1.  ]])

In [13]:
import cv2

In [14]:
# Four corners in source image
pts_src = np.array([[684, 530], [728, 529], [685, 574],[728, 572]])

# Four corners in destination image
pts_dst = np.array([[661, 591],[704, 592],[660, 633],[702, 634]])

# Calculate Homography
h, status = cv2.findHomography(pts_src, pts_dst)

In [15]:
h

array([[ 4.30592128e-01, -2.07816394e-02,  2.00839446e+02],
       [-2.06908950e-01,  7.09575758e-01,  1.98508004e+02],
       [-4.04476587e-04,  1.77654352e-05,  1.00000000e+00]])